# Source :
https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/cornac_bivae_deep_dive.ipynb

In [49]:
import numpy as np
import pandas as pd


In [50]:
#Neuropsychology 피처
df = pd.read_csv('merge_df.csv')
ne = ['소요시간',
 'RT_A3',
 'RT_A4',
 'RT_A5',
 'RT_A8',
 'RT_A10',
 '소요시간.1',
 'RT_B1',
 'RT_B2',
 'RT_B3',
 'RT_B5',
 'RT_B9',
 'RT_A_median',
 'RT_A_sd',
 'RT_A_min',
 'RT_A_max',
 'RT_B_sum',
 'RT_B_median',
 'RT_B_sd',
 'RT_B_min',
 'RT_B_max',
 '정답수.2',
 'cal10.1',
 'RT_cal1',
 'RT_cal2',
 'RT_cal3',
 'RT_cal4',
 'RT_cal6',
 'RT_cal7',
 'RT_cal8',
 'RT_cal9',
 'RT_cal10',
 'RT_cal12',
 'RT_cal_sum',
 'RT_cal_median',
 '소요시간.2',
 '정답수.3',
 'RT_sa1',
 'RT_sa2',
 'RT_sa3',
 'RT_sa4',
 'RT_sa5',
 'RT_sa6',
 'RT_sa_sum',
 'RT_sa_mean',
 'RT_sa_sd',
 'RT_sa_min',
 'RT_sa_max',
 '소요시간.5',
 '소요시간.6',
 '소요시간.7', 'Key']
n_df = df[ne]

In [51]:
#SP 결측치처리
ls = []

for i in df.columns:
    if 'SP' in i:
        ls.append(i)
        
sp_df = df[ls]

m_df = pd.concat([n_df, sp_df], axis=1)

m_df.isnull().sum()
m_df_key = m_df['Key']
m_df = m_df.drop('Key',axis=1)

In [52]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1.0, 5.0))
scaler_df = scaler.fit_transform(m_df)
m_df = pd.DataFrame(scaler_df, columns = m_df.columns)

In [53]:
dict = {}
for index, row in m_df.iterrows():
    dict[index] = row.tolist()

In [54]:
new_df = pd.DataFrame(index=range(0,len(m_df.columns)*len(m_df)), columns=['user_id','item_id','target'])
i = 0
for key, value in dict.items():
    item = 0
    for v in value:
        new_df['user_id'][i] = key
        new_df['item_id'][i] = item
        new_df['target'][i] = v
        item += 1
        i += 1
new_df = new_df.astype({'user_id':'int'})
new_df = new_df.astype({'item_id':'int'})
new_df = new_df.astype({'target':'float'})

new_df

,user_id,item_id,target
0,0,0,1.385712
1,0,1,1.504136
2,0,2,1.477716
3,0,3,1.258207
4,0,4,1.541219
...,...,...,...
27739,288,91,1.844359
27740,288,92,1.627855
27741,288,93,1.627855
27742,288,94,1.115004


In [55]:
new_df = new_df.sample(frac=1).reset_index(drop=True)

In [56]:
train_df = new_df.dropna(axis=0)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27517 entries, 0 to 27743
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  27517 non-null  int32  
 1   item_id  27517 non-null  int32  
 2   target   27517 non-null  float64
dtypes: float64(1), int32(2)
memory usage: 644.9 KB


In [57]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.models.cornac.cornac_utils import predict
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 5
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 128
LEARNING_RATE = 0.001

System version: 3.9.17 (main, Jul  5 2023, 20:47:11) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.0.1+cpu
Cornac version: 1.16


In [58]:
##
train_set = cornac.data.Dataset.from_uir(train_df.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 289
Number of items: 96


In [59]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 9.7121 seconds for training.


 predict(
    model,
    data,
    usercol=DEFAULT_USER_COL,
    itemcol=DEFAULT_ITEM_COL,
    predcol=DEFAULT_PREDICTION_COL,
):

In [60]:
from recommenders.models.cornac.cornac_utils import predict
with Timer() as t:
    all_predictions2 = predict(bivae, train_df, usercol='user_id', itemcol='item_id', predcol='target')
print("Took {} seconds for prediction.".format(t))

Took 1.8190 seconds for prediction.


In [61]:
all_predictions2

,user_id,item_id,target
0,200,3,[4.9250956]
1,169,75,[4.922434]
2,154,95,[4.919133]
3,180,74,[4.922434]
4,146,50,[4.9250956]
...,...,...,...
27512,195,9,[4.9250956]
27513,35,63,[4.922434]
27514,136,6,[4.9250956]
27515,190,52,[4.9206085]


In [63]:
r_df = pd.DataFrame(index=range(289), columns=range(96))

for i in range(len(all_predictions2)):
    row = all_predictions2.loc[i, 'user_id']
    col = all_predictions2.loc[i, 'item_id']
    value = all_predictions2.loc[i, 'target']
    r_df.loc[row, col] = value
original_values = scaler.inverse_transform(r_df)
original_df = pd.DataFrame(original_values, columns = m_df.columns)
original_df

,소요시간,RT_A3,RT_A4,RT_A5,RT_A8,RT_A10,소요시간.1,RT_B1,RT_B2,RT_B3,...,SP3.Gain:std,SP3.Gain:max,SP3.InitAccel:median,SP3.InitAccel:std,SP3.InitAccel:min,SP3.InitAccel:max,SP3.Velocity:mean,SP3.Velocity:median,SP3.Velocity:std,SP3.Velocity:min
0,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
1,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
2,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
3,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
4,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
285,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
286,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
287,57.78646,10.219653,12.984609,14.696052,7.24261,5.750564,81.517989,28.75265,40.369978,30.599434,...,6.726718,10.652581,0.269949,0.215527,0.268631,0.270617,5.526889,5.526889,6.295478,3.290988
